In [5]:
##NOTE: Install All the Required Dependencies If Not Yet Installed on Your Device
#!pip3 install torch torchvision torchaudio
#!pip install transformers ipywidgets gradio --upgrade
#!pip install --upgrade transformers accelerate
#!pip install --upgrade gradio
#!pip install nltk
#!pip install jiwer
#!pip install sentencepiece
#!pip install sacremoses
#!pip install soundfile
#!pip install librosa numpy jiwer nltk
#!pip install --upgrade pip 
#!pip install huggingface_hub

In [6]:
#Import Required Libraries
from transformers import pipeline
from jiwer import wer
from transformers import VitsModel, AutoTokenizer, set_seed
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import soundfile as sf
import librosa
from scipy.spatial.distance import euclidean
import numpy as np
import string
import os
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
import string
import numpy as np
import librosa
from scipy.spatial.distance import euclidean
import string

In [7]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional if using WordNet's multilingual features
import nltk
print(nltk.data.path)
import nltk
nltk.data.path.append('./nltk_data')

['/home/ai4cu/nltk_data', '/opt/tljh/user/nltk_data', '/opt/tljh/user/share/nltk_data', '/opt/tljh/user/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', './nltk_data']


[nltk_data] Downloading package wordnet to /home/ai4cu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ai4cu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
def translate_transformers(model_name, lang_name, source_text, device=0):
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(f"cuda:{device}" if torch.cuda.is_available() else "cpu")

    # Handle model-specific config
    if "nllb" in model_name.lower():
        #Facebook NLLB-200-3.3B
        #Switch to Selected Language
        if lang_name == "Yoruba":
            tgt_lang_token = "yor_Latn"
        elif lang_name == "Hausa":
            tgt_lang_token = "hau_Latn"
        elif lang_name == "Igbo":
            tgt_lang_token = "ibo_Latn"
        elif lang_name == "French":
            tgt_lang_token = "fra_Latn"
        else:
            raise ValueError("Unsupported target language. Choose from 'Yoruba', 'Hausa', 'Igbo'.")   
        tokenizer.src_lang = "eng_Latn"
        forced_bos_token_id = tokenizer.convert_tokens_to_ids(tgt_lang_token)
        inputs = tokenizer(source_text, return_tensors="pt").to(model.device)
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=forced_bos_token_id,
            max_length=1000,
        )

    elif "m2m100" in model_name.lower() and lang_name == "Yoruba":
        # Davlan m2m100_418M-eng-yor-mt
        tokenizer.src_lang = "en"
        tgt_lang_token = "yo"
        inputs = tokenizer(source_text, return_tensors="pt").to(model.device)

        model.config.forced_bos_token_id = tokenizer.lang_code_to_id[tgt_lang_token]

        outputs = model.generate(
            **inputs,
            max_length=1000,
        )

    else:
        raise ValueError("Unsupported MT Model or Language. Please Use facebook/nllb-200-3.3B(Supports 200 Languages) or Davlan/m2m100_418M-eng-yor-mt(Finetuned for Yoruba).")

    translated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return translated_text

In [9]:
def reset_fields():
    return "","","",""  # Clear modelName, langName, inputText, and audio output

In [11]:
#Define User Interface Function using Gradio and IPython Libraries
import gradio as gr
from IPython.display import Audio

with gr.Blocks() as interface:
    #gr.Markdown(description)
    with gr.Row():
       modelName = gr.Dropdown(["facebook/nllb-200-3.3B",
                                 "Davlan/m2m100_418M-eng-yor-mt"],
                               label="Select Text2Text Translation Model", 
                               allow_custom_value=True)
    with gr.Row():
       langName = gr.Dropdown(["Yoruba",#Lang1
                               "Hausa", #Lang2
                               "Igbo",  #Lang3
                               "French" #Lang4
                               ], 
                               label="Select Target Language",
                               allow_custom_value=True)
    with gr.Row():
        inputText = gr.Textbox(placeholder="Enter English Text Here...",label="Input Text", lines=6)
    with gr.Row():
        btn = gr.Button("Translate")
    with gr.Row():
        #output = gr.Audio(type="filepath", label="Click Play/Pause to Generate Speech for Current Text and Reset Button for New Language Selection/Text Input")
        outputText = gr.Textbox(value="Translated Text Will Appear Here...", label="Translated Text")
    with gr.Row():
        reset_btn = gr.Button("Reset")
    btn.click(
        fn=translate_transformers,
        inputs= [modelName, langName, inputText],
        outputs=outputText,
    )
    reset_btn.click(
        fn=reset_fields,
        inputs=[],
        outputs=[modelName, langName, inputText, outputText],
    )
interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://5ea1500fe6e15025e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]